In [32]:
import pandas as pd

In [33]:
df = pd.read_json('latest_grocery_data.json')

In [34]:
df.columns

Index(['id', 'concatted', 'vendor', 'product_name', 'units', 'brand',
       'detail_url', 'sku', 'upc', 'product_id', 'current_price', 'old_price',
       'price_per_unit', 'other'],
      dtype='object')

In [36]:
df = df.drop(
    columns=[
        "concatted",
        "old_price",
        "price_per_unit",
        "sku",
        "upc",
        "product_id",
        "old_price",
        "other",
    ],
)

In [37]:
df = df.dropna()

In [39]:
df = df.drop_duplicates()

In [40]:
# Split units into quantity + unit
units_split = df['units'].str.extract(r'(?P<unit_quantity>\d+(?:\.\d+)?)\s*(?P<unit>[a-zA-Z]+)')
df = df.assign(**units_split)
df[['units', 'unit_quantity', 'unit']].head()

,units,unit_quantity,unit
57002,500g,500,g
57004,$32.90/1kg,1,kg
57006,$55.00/1kg,1,kg
57007,$29.90/1kg,1,kg
57009,$6.59/1kg $2.99/1lb,1,kg


In [41]:
df

,id,vendor,product_name,units,brand,detail_url,current_price,unit_quantity,unit
57002,10877324,Loblaws,Gummy Sour Worms,500g,Carnaby Sweet,https://www.loblaws.ca/gummy-sour-worms/p/2136...,3.99,500,g
57004,10877816,Loblaws,Hot Genoa,$32.90/1kg,PC Splendido,https://www.loblaws.ca/hot-genoa/p/21049250_KG...,3.29,1,kg
57006,10889620,Loblaws,Bourbon Walnuts,$55.00/1kg,Johnvince Foods,https://www.loblaws.ca/bourbon-walnuts/p/21564...,5.5,1,kg
57007,10894017,Loblaws,German Salami (Thin Sliced),$29.90/1kg,Pillers,https://www.loblaws.ca/german-salami-thin-slic...,2.99,1,kg
57009,10894358,Loblaws,Grade A Frozen Turkey,$6.59/1kg $2.99/1lb,Butterball,https://www.loblaws.ca/grade-a-frozen-turkey/p...,65.9,1,kg
...,...,...,...,...,...,...,...,...,...
228428,41514057,Walmart,"Raspberry Turnover Cookies, 300 g",300g,Voortman,https://www.walmart.ca/en/ip/Voortman-Raspberr...,2.98,300,g
228429,41514058,Walmart,"Marshmallow Treats Bars, 120 g, 5 Bars",5 bars,Lucky Charms,https://www.walmart.ca/en/ip/Lucky-Charms-Mars...,3.47,5,bars
228430,41514059,Walmart,"SmartsSweets, Sweet Fish, 50g Pouch, Candy wit...",candy with no artificial sweeteners or added s...,SmartSweets,https://www.walmart.ca/en/ip/smartssweets-swee...,4.48,NaN,NaN
228431,41514060,Walmart,NESTLÉ Candy Coated Milk Chocolate Carton 45 g...,45 gr,Smarties,https://www.walmart.ca/en/ip/nestl-smarties-ca...,0.98,45,gr


In [43]:
import re

# Heuristic aisle assignment based on product_name keywords
AISLE_RULES = [
    ('Produce', [
        r'\bapple(s)?\b', r'\bbanana(s)?\b', r'\bavocado(s)?\b', r'\bberry(ies)?\b',
        r'\bgrape(s)?\b', r'\bcitrus\b', r'\blemon(s)?\b', r'\blime(s)?\b',
        r'\borange(s)?\b', r'\btomato(es)?\b', r'\bonion(s)?\b', r'\bgarlic\b',
        r'\blettuce\b', r'\bspinach\b', r'\bcarrot(s)?\b', r'\bcucumber(s)?\b',
        r'\bpepper(s)?\b', r'\bbroccoli\b', r'\bcauliflower\b', r'\bpotato(es)?\b',
        r'\bmushroom(s)?\b', r'\bherb(s)?\b', r'\bcilantro\b', r'\bparsley\b',
        r'\bbasil\b', r'\bsalad\b', r'\bfruit\b', r'\bvegetable(s)?\b',
    ]),
    ('Dairy', [
        r'\bmilk\b', r'\bcheese\b', r'\byogurt\b', r'\bbutter\b', r'\bcream\b',
        r'\bcheddar\b', r'\bmozzarella\b', r'\bparmesan\b', r'\bcottage cheese\b',
        r'\bsour cream\b', r'\bwhipped\b', r'\bhalf\s*&\s*half\b', r'\bkefir\b',
        r'\begg(s)?\b',
    ]),
    ('Meat', [
        r'\bbeef\b', r'\bpork\b', r'\bchicken\b', r'\bturkey\b', r'\bsausage\b',
        r'\bbacon\b', r'\bham\b', r'\bsteak\b', r'\bground\b', r'\bsalami\b',
        r'\bpepperoni\b', r'\blamb\b', r'\bseafood\b', r'\bshrimp\b', r'\bfish\b',
        r'\bsalmon\b', r'\btuna\b', r'\bcod\b', r'\bcrab\b', r'\blobster\b',
        r'\bdeli\b', r'\bprosciutto\b',
    ]),
    ('Frozen', [
        r'\bfrozen\b', r'\bice cream\b', r'\bgelato\b', r'\bpopsicle(s)?\b',
        r'\bice pop(s)?\b', r'\bfrozen pizza\b', r'\bfries\b', r'\bnugget(s)?\b',
    ]),
    ('Beverages', [
        r'\bsoda\b', r'\bjuice\b', r'\bwater\b', r'\btea\b', r'\bcoffee\b',
        r'\bbeer\b', r'\bwine\b', r'\bcider\b', r'\bkombucha\b', r'\bsparkling\b',
        r'\bdrink(s)?\b', r'\bcola\b', r'\benergy\b', r'\bsports drink\b',
    ]),
    ('Bakery', [
        r'\bbread\b', r'\bbagel(s)?\b', r'\bbun(s)?\b', r'\broll(s)?\b',
        r'\bcroissant(s)?\b', r'\bmuffin(s)?\b', r'\bcake\b', r'\bcookie(s)?\b',
        r'\bpastry\b', r'\bdonut(s)?\b', r'\btortilla(s)?\b', r'\bpita\b', r'\bnaan\b',
    ]),
    ('Snacks', [
        r'\bchip(s)?\b', r'\bcracker(s)?\b', r'\bpopcorn\b', r'\bpretzel(s)?\b',
        r'\btrail mix\b', r'\bgranola bar(s)?\b', r'\bprotein bar(s)?\b',
        r'\bsnack(s)?\b', r'\bnut(s)?\b', r'\bcandy\b', r'\bchocolate\b',
    ]),
    ('Pantry', [
        r'\bpasta\b', r'\brice\b', r'\bflour\b', r'\bsugar\b', r'\bsalt\b',
        r'\bspice(s)?\b', r'\boil\b', r'\bvinegar\b', r'\bsauce\b', r'\bketchup\b',
        r'\bmustard\b', r'\bmayonnaise\b', r'\bcereal\b', r'\boats\b', r'\bbean(s)?\b',
        r'\blentil(s)?\b', r'\bsoup\b', r'\bbroth\b', r'\bcanned\b', r'\bjar\b',
        r'\bcan\b', r'\bpackaged\b',
    ]),
]

def assign_aisle(name):
    if not isinstance(name, str):
        return 'Other'
    text = name.lower()
    for aisle, patterns in AISLE_RULES:
        for pattern in patterns:
            if re.search(pattern, text):
                return aisle
    return 'Other'

df['aisle'] = df['product_name'].apply(assign_aisle)
df['aisle'].value_counts().head()


aisle
Other      36362
Produce    25654
Dairy      23331
Meat       13790
Pantry     13431
Name: count, dtype: int64

In [44]:
df.to_json('cleaned_grocery_data.json', orient='records', lines=True)